In [1]:
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import patsy


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

In [2]:
#  Abrindo o dataset SAM
df = pd.read_excel(r"C:\Users\sabri\SAMTeams.xlsx")
df.head()

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals,assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,0848a777-a0fc-4fa0-ac5a-97c1d420c764,SAM N PBH VS TN G2 2022-11-04.17.04,Aquadome,2022-11-04,loss,PALITO,TRUE NEUTRAL,756,0,0,4,5,6,0.000000,1,2,0.666667
1,0848a777-a0fc-4fa0-ac5a-97c1d420c764,SAM N PBH VS TN G2 2022-11-04.17.04,Aquadome,2022-11-04,win,TRUE NEUTRAL,PALITO,980,1,1,4,6,5,16.666667,2,1,0.800000
2,2a1f54d6-a34f-4729-9368-294821fbdcff,SAM V EXD VS MPYR G4 2022-11-04.18.14,Utopia Coliseum (Dusk),2022-11-04,win,EXEED BRAZIL,EMPYRE,987,2,2,2,10,3,20.000000,2,3,0.666667
3,2a1f54d6-a34f-4729-9368-294821fbdcff,SAM V EXD VS MPYR G4 2022-11-04.18.14,Utopia Coliseum (Dusk),2022-11-04,loss,EMPYRE,EXEED BRAZIL,766,0,0,5,3,10,0.000000,3,2,0.500000
4,d37eaf4e-f01a-4d5d-b6b2-c029b63ca100,SAM Z OUT vs EXD G3 2022-11-04.17.04,DFH Stadium,2022-11-04,win,ONLY OUTLIERS,EXEED BRAZIL,976,4,2,1,8,2,50.000000,7,3,0.500000


In [3]:
#  Renomear a coluna assists para goals_assists e goals para goals_solo
df.rename(columns={'assists':'goals_assists'}, inplace = True)
df.rename(columns={'goals':'goals_solo'}, inplace = True)
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,0848a777-a0fc-4fa0-ac5a-97c1d420c764,SAM N PBH VS TN G2 2022-11-04.17.04,Aquadome,2022-11-04,loss,PALITO,TRUE NEUTRAL,756,0,0,4,5,6,0.000000,1,2,0.666667
1,0848a777-a0fc-4fa0-ac5a-97c1d420c764,SAM N PBH VS TN G2 2022-11-04.17.04,Aquadome,2022-11-04,win,TRUE NEUTRAL,PALITO,980,1,1,4,6,5,16.666667,2,1,0.800000
2,2a1f54d6-a34f-4729-9368-294821fbdcff,SAM V EXD VS MPYR G4 2022-11-04.18.14,Utopia Coliseum (Dusk),2022-11-04,win,EXEED BRAZIL,EMPYRE,987,2,2,2,10,3,20.000000,2,3,0.666667
3,2a1f54d6-a34f-4729-9368-294821fbdcff,SAM V EXD VS MPYR G4 2022-11-04.18.14,Utopia Coliseum (Dusk),2022-11-04,loss,EMPYRE,EXEED BRAZIL,766,0,0,5,3,10,0.000000,3,2,0.500000
4,d37eaf4e-f01a-4d5d-b6b2-c029b63ca100,SAM Z OUT vs EXD G3 2022-11-04.17.04,DFH Stadium,2022-11-04,win,ONLY OUTLIERS,EXEED BRAZIL,976,4,2,1,8,2,50.000000,7,3,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,8ce84187-c75d-46f7-b3d1-d97d5173ce69,SAM AA w7m vs KR G4 2023-06-10.16.30,Utopia Coliseum (Dusk),2023-06-10,loss,W7M ESPORTS,KRu ESPORTS,928,1,1,6,2,12,50.000000,3,4,0.500000
2610,18861644-a61a-4656-abf0-64dfc92ad55a,SAM AD TS vs KR G4 2023-06-11.17.50,Utopia Coliseum (Dusk),2023-06-11,loss,TEAM SECRET,KRu ESPORTS,573,0,0,3,6,7,0.000000,1,3,0.428571
2611,18861644-a61a-4656-abf0-64dfc92ad55a,SAM AD TS vs KR G4 2023-06-11.17.50,Utopia Coliseum (Dusk),2023-06-11,win,KRu ESPORTS,TEAM SECRET,1134,3,2,5,7,6,42.857143,3,1,0.833333
2612,3a2724fe-da3d-47ee-9cbc-d4013fa26538,SAM U NOLA vs BZR G2 2023-06-09.18.16,Forbidden Temple,2023-06-09,loss,BZR ESPORTS,NOLA,1844,1,1,15,7,22,14.285714,5,4,0.681818


In [4]:
#  Consertando os valores da variável goals solo para excluir os valores de goals_assists
df['goals_solo'] = df['goals_solo'] - df['goals_assists']
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,0848a777-a0fc-4fa0-ac5a-97c1d420c764,SAM N PBH VS TN G2 2022-11-04.17.04,Aquadome,2022-11-04,loss,PALITO,TRUE NEUTRAL,756,0,0,4,5,6,0.000000,1,2,0.666667
1,0848a777-a0fc-4fa0-ac5a-97c1d420c764,SAM N PBH VS TN G2 2022-11-04.17.04,Aquadome,2022-11-04,win,TRUE NEUTRAL,PALITO,980,0,1,4,6,5,16.666667,2,1,0.800000
2,2a1f54d6-a34f-4729-9368-294821fbdcff,SAM V EXD VS MPYR G4 2022-11-04.18.14,Utopia Coliseum (Dusk),2022-11-04,win,EXEED BRAZIL,EMPYRE,987,0,2,2,10,3,20.000000,2,3,0.666667
3,2a1f54d6-a34f-4729-9368-294821fbdcff,SAM V EXD VS MPYR G4 2022-11-04.18.14,Utopia Coliseum (Dusk),2022-11-04,loss,EMPYRE,EXEED BRAZIL,766,0,0,5,3,10,0.000000,3,2,0.500000
4,d37eaf4e-f01a-4d5d-b6b2-c029b63ca100,SAM Z OUT vs EXD G3 2022-11-04.17.04,DFH Stadium,2022-11-04,win,ONLY OUTLIERS,EXEED BRAZIL,976,2,2,1,8,2,50.000000,7,3,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,8ce84187-c75d-46f7-b3d1-d97d5173ce69,SAM AA w7m vs KR G4 2023-06-10.16.30,Utopia Coliseum (Dusk),2023-06-10,loss,W7M ESPORTS,KRu ESPORTS,928,0,1,6,2,12,50.000000,3,4,0.500000
2610,18861644-a61a-4656-abf0-64dfc92ad55a,SAM AD TS vs KR G4 2023-06-11.17.50,Utopia Coliseum (Dusk),2023-06-11,loss,TEAM SECRET,KRu ESPORTS,573,0,0,3,6,7,0.000000,1,3,0.428571
2611,18861644-a61a-4656-abf0-64dfc92ad55a,SAM AD TS vs KR G4 2023-06-11.17.50,Utopia Coliseum (Dusk),2023-06-11,win,KRu ESPORTS,TEAM SECRET,1134,1,2,5,7,6,42.857143,3,1,0.833333
2612,3a2724fe-da3d-47ee-9cbc-d4013fa26538,SAM U NOLA vs BZR G2 2023-06-09.18.16,Forbidden Temple,2023-06-09,loss,BZR ESPORTS,NOLA,1844,0,1,15,7,22,14.285714,5,4,0.681818


In [5]:
#  Checando se há empates nos resultados e removendo-os
df_draw = df[df['result']=='draw']
df = df.drop(df_draw.index)

In [6]:
#  Transformando result em variável binária. Loss = 1 e win = 0
df1 = pd.get_dummies(df['result'])
#df1.head()
df1 = df1.drop(['win'], axis = 1)
#df1.head()
df = pd.concat((df, df1), axis = 1)
#df.head()
df = df.drop(['result'], axis = 1)
df = df.rename(columns = {'loss': 'result'})
df.head()

,replay_id,replay_title,map,date,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded,result
0,0848a777-a0fc-4fa0-ac5a-97c1d420c764,SAM N PBH VS TN G2 2022-11-04.17.04,Aquadome,2022-11-04,PALITO,TRUE NEUTRAL,756,0,0,4,5,6,0.000000,1,2,0.666667,1
1,0848a777-a0fc-4fa0-ac5a-97c1d420c764,SAM N PBH VS TN G2 2022-11-04.17.04,Aquadome,2022-11-04,TRUE NEUTRAL,PALITO,980,0,1,4,6,5,16.666667,2,1,0.800000,0
2,2a1f54d6-a34f-4729-9368-294821fbdcff,SAM V EXD VS MPYR G4 2022-11-04.18.14,Utopia Coliseum (Dusk),2022-11-04,EXEED BRAZIL,EMPYRE,987,0,2,2,10,3,20.000000,2,3,0.666667,0
3,2a1f54d6-a34f-4729-9368-294821fbdcff,SAM V EXD VS MPYR G4 2022-11-04.18.14,Utopia Coliseum (Dusk),2022-11-04,EMPYRE,EXEED BRAZIL,766,0,0,5,3,10,0.000000,3,2,0.500000,1
4,d37eaf4e-f01a-4d5d-b6b2-c029b63ca100,SAM Z OUT vs EXD G3 2022-11-04.17.04,DFH Stadium,2022-11-04,ONLY OUTLIERS,EXEED BRAZIL,976,2,2,1,8,2,50.000000,7,3,0.500000,0


In [7]:
#  Excluindo várias colunas de uma vez
df.drop(['replay_id','replay_title','map','team_name','opposing_team_name', 'shots_conceded'], axis = 1, inplace = True)

In [8]:
#  Apagando as observações vazias
df.dropna(axis = 0, inplace = True)

In [9]:
#  Dividindo a base de validação
periodo_des = df[df['date'] < datetime(2023,6,1)].copy()
oot = df[df['date'] > datetime(2023,6,1)].copy()
periodo_des.reset_index(inplace = True)
oot.reset_index(inplace=True)

print(periodo_des.shape, oot.shape)

(2362, 12) (250, 12)


In [10]:
#  Definindo x e y
x_oot = oot.drop(['result', 'date', 'index'], axis = 1)
y_oot = oot['result']
x = periodo_des.drop(['result', 'date', 'index'], axis = 1)
y = periodo_des['result']
x_oot.shape

(250, 9)

In [11]:
#  Separando a base de desenvolvimento em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
x_train.shape, x_test.shape

((1771, 9), (591, 9))

In [12]:
#  Definindo o modelo de trabalho - Gradient Boosting

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

#  Predict
# predict the mode
y_pred = model.predict(x_test)
classification_report(y_test, y_pred)
# performance evaluation metrics
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.89      0.88      0.89       288
           1       0.89      0.89      0.89       303

    accuracy                           0.89       591
   macro avg       0.89      0.89      0.89       591
weighted avg       0.89      0.89      0.89       591



In [13]:
#  Definindo parâmetros
param_grid = {'n_estimators': [10, 100, 300, 600] 
              , 'min_samples_leaf': [1, 10, 20, 50]
              , 'learning_rate': [.05, .1, .25, .5, 1]
             }

In [14]:
#  Separando os valores de cv
#  **parâmetro CV:** Significa *cross validation* (ou validação cruzada). 
#  Nele indicamos os índices das observações que serão utilizadas para desenvolver o modelo e para validar respectivamente. 
#  Através deste parâmetro, a função ainda consegue fazer outras modalidades de validação cruzada.

cv = KFold(n_splits=10, shuffle=True, random_state=10)

In [15]:
#  Fazendo a procura dos melhores parâmetros no modelo
grid_search = GridSearchCV(model,
                           param_grid=param_grid,
                          scoring = 'roc_auc',
                          cv = cv)

grid_search.fit(x, y)
print(grid_search.best_estimator_)

GradientBoostingClassifier(learning_rate=0.05, min_samples_leaf=10,
                           n_estimators=300)


In [19]:
#  Aplicar melhores resultados aqui
model = GradientBoostingClassifier(min_samples_leaf = 10, learning_rate = 0.05, n_estimators = 300)
model.fit(x_train, y_train)


# predict the mode
y_pred = model.predict(x_test)
av1 = classification_report(y_test, y_pred)
y_pred1 = model.predict(x_oot)
av2 = classification_report(y_oot, y_pred1)
# performance evaluatio metrics
print(av1, av2)

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       286
           1       0.89      0.89      0.89       305

    accuracy                           0.89       591
   macro avg       0.89      0.89      0.89       591
weighted avg       0.89      0.89      0.89       591
               precision    recall  f1-score   support

           0       0.90      0.89      0.90       125
           1       0.89      0.90      0.90       125

    accuracy                           0.90       250
   macro avg       0.90      0.90      0.90       250
weighted avg       0.90      0.90      0.90       250



In [20]:
#  Avaliando modelo na base de treino
y_pred0 = model.predict(x_train)
av0 = classification_report(y_train, y_pred0)
print(av0)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       895
           1       0.94      0.94      0.94       876

    accuracy                           0.94      1771
   macro avg       0.94      0.94      0.94      1771
weighted avg       0.94      0.94      0.94      1771



In [21]:
#  Separando as features para identificação da importância das variáveis
feature_list = list(x.columns)
#  Importância das variáveis
importances = list(model.feature_importances_)
# Print out the feature and importances 
print(feature_list, importances)

['score', 'goals_solo', 'goals_assists', 'saves', 'shots', 'shooting_percentage', 'demos_inflicted', 'demos_taken', 'save_conceded'] [0.040597268441557456, 0.1233566501365183, 0.32980901479171426, 0.0747335815477275, 0.038506673774504216, 0.1160902036303492, 0.006468771904485516, 0.013620341385562762, 0.25681749438758084]
